<a href="https://colab.research.google.com/github/YoheiFukuhara/keras-for-beginner/blob/master/Keras02_SaveModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[「【Keras入門(2)】訓練モデル保存(KerasモデルとSavedModel)」](https://qiita.com/FukuharaYohei/items/ac6333391b8abda94bdc)で解説しています。

[公式チュートリアル](https://www.tensorflow.org/tutorials/keras/save_and_restore_models)を参考にしました。



In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.models import load_model

#tf.enable_eager_execution()

# テストデータの作成


## 説明変数
0から1までの乱数で128, 2の配列を作成

In [0]:
NUM_TRAIN = 128
data = np.random.rand(NUM_TRAIN,2)

## 目的変数
説明変数の各行の2つの値を足した数

In [0]:
labels = (np.sum(data, axis=1) > 1.0) * 1
labels = labels.reshape(NUM_TRAIN,1)

# モデル定義

In [0]:
# Sequentialモデル使用(Sequentialモデルはレイヤを順に重ねたモデル)
model = Sequential()

# 全結合層(2層->4層)
model.add(Dense(4, input_dim=2, activation="tanh"))

# 結合層(4層->1層)：入力次元を省略すると自動的に前の層の出力次元数を引き継ぐ
model.add(Dense(1, activation="sigmoid"))

# モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


#訓練

In [0]:
# Callbackを定義し、モデル保存の追加
li_cb = []
li_cb.append(ModelCheckpoint('./model.hdf5', save_best_only=True))

In [0]:
model.fit(data, labels, epochs=300, validation_split=0.2, callbacks=li_cb)

Train on 102 samples, validate on 26 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
102/102 [==============================] - 0s 4ms/sample - loss: 0.6685 - acc: 0.5294 - val_loss: 0.6748 - val_acc: 0.5000
Epoch 2/300
102/102 [==============================] - 0s 268us/sample - loss: 0.6674 - acc: 0.5294 - val_loss: 0.6740 - val_acc: 0.4615
Epoch 3/300
102/102 [==============================] - 0s 213us/sample - loss: 0.6665 - acc: 0.5294 - val_loss: 0.6731 - val_acc: 0.5000
Epoch 4/300
102/102 [==============================] - 0s 216us/sample - loss: 0.6659 - acc: 0.5294 - val_loss: 0.6725 - val_acc: 0.4615
Epoch 5/300
102/102 [==============================] - 0s 197us/sample - loss: 0.6645 - acc: 0.5294 - val_loss: 0.6716 - val_acc: 0.5000
Epoch 6/300
102/102 [==============================] - 0s 210us/sample - loss: 0.6638 - acc: 0.5294 - val_loss: 0.6706 - val_acc: 0.5000
Epoch 7/300
102/102 [==============================] - 0s 214us/sample - loss: 0.6629 -

In [0]:
# SavedModelフォーマットで保存(TensorFlow2.0ではexport_saved_modelを使う
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/experimental/export_saved_model
tf.contrib.saved_model.save_keras_model(model, './models/keras_export')

# Eager Executionでないとserving_only=Trueは失敗
#tf.contrib.saved_model.save_keras_model(model, './models/keras_export', serving_only=True)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensor

b'./models/keras_export/1558645955'

In [0]:
# フォルダ名が動的に生成されるので、都度変更が必要
!saved_model_cli show --all --dir ./models/keras_export/1558645955


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: init_1
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: dense_1/Sigmoid:0
  Method name is: tensorflow/serving/predict


# テスト実行

In [0]:
NUM_TEST = 50
test_data = np.random.rand(NUM_TEST,2)
test_labels = (np.sum(test_data, axis=1) > 1.0) * 1

In [0]:
predict = ((model.predict(test_data) > 0.5) * 1).reshape(NUM_TEST)
print(predict)
print(test_labels)
print("Accuracy:",sum(predict == test_labels) / NUM_TEST)

[1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 1 1 1
 0 0 1 0 1 1 0 0 1 0 1 0 0]
[1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1
 0 0 1 0 1 0 0 0 1 0 1 0 0]
Accuracy: 0.88


#Keras モデルをロードしてSavedModelフォーマットで保存

In [0]:
# モデルを初期化してロード
model = None
model = load_model('./model.hdf5')

In [0]:
tf.contrib.saved_model.save_keras_model(model, './models/keras_export')

# Eager Executionでないとserving_only=Trueは失敗
#tf.contrib.saved_model.save_keras_model(model, './models/keras_export', serving_only=True)


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./models/keras_export/1558646001/saved_model.pb


b'./models/keras_export/1558646001'

In [0]:
# ライブラリバージョン確認
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.0.1
astor==0.7.1
astropy==3.0.5
atari-py==0.1.7
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.7
autograd==1.2
Babel==2.6.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
bokeh==1.0.4
boto==2.49.0
boto3==1.9.150
botocore==1.12.150
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.0
certifi==2019.3.9
cffi==1.12.3
chainer==5.4.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cvxopt==1.2.3
cvxpy==1.0.15
cycler==0.10.0
cymem==2.0.2
Cython==0.29.7
cytoolz==0.9.0.1
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.6.0
dill==0.2.9
distributed==1.25.3
Django==2.2.1
dlib==19.16.0
dm-sonnet==1.32
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
easydict==1.9
ecos=

In [0]:
# フォルダ名が動的に生成されるので、都度変更が必要
!saved_model_cli show --all --dir ./models/keras_export/1558646001


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: init_1
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: dense_1/Sigmoid:0
  Method name is: tensorflow/serving/predict
